In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Grounding with Gemini in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fintro-grounding-gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner), [Kristopher Overholt](https://github.com/koverholt) |

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

## Overview

[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) lets you use generative text models to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in Google Search results or data stores within [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

In the context of grounding in Vertex AI, you can configure two different sources of grounding:

1. Google Search results for data that is publicly available and indexed
2. [Data stores in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest), which can include your own data in the form of website data, unstructured data, or structured data

### Allowlisting

Some of the features in this sample notebook require access to certain features via an allowlist. [Grounding with Vertex AI Search](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) is available in Public Preview, whereas Grounding with Google Search results is generally available.

If you use this service in a production application, you will also need to [use a Google Search entry point](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points).

### Objective

In this tutorial, you learn how to:

- Generate LLM text and chat model responses grounded in Google Search results
- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, web grounding, data store grounding)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable the [Vertex AI and Vertex AI Agent Builder APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,discoveryengine.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Installation

Install the following packages required to execute this notebook.

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


Restart the kernel after installing packages:

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your Google Cloud account

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [1]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
PROJECT_ID = "qwiklabs-gcp-04-bde50cd42a87"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

In [3]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

### Import libraries

In [4]:
from IPython.display import Markdown, display
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)
from vertexai.preview.generative_models import grounding as preview_grounding

Initialize the Gemini model from Vertex AI:

In [5]:
model = GenerativeModel("gemini-1.5-pro")

## Helper functions

In [6]:
def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[{grounding_chunk_index + 1}]"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"

    display(Markdown(markdown_text))

## Example: Grounding with Google Search results

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search. You'll ask a question about a recent hardware release from the Google Store.

In [7]:
PROMPT = "You are an expert in astronomy. When is the next solar eclipse in the US?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [8]:
response = model.generate_content(PROMPT)

display(Markdown(response.text))

I can't give you the date of the next solar eclipse in the US.  I can generate human-like text in response to a wide range of prompts and questions, but my knowledge about upcoming astronomical events is limited. 

**To find the date of the next solar eclipse, I recommend checking these resources:**

* **NASA Eclipse Website:** This website has a comprehensive list of upcoming eclipses, including dates, times, and locations.
* **Time and Date Website:** This website provides detailed information about eclipses, including interactive maps and visibility information.
* **Astronomy Magazines and Websites:** Publications like Astronomy Magazine or Sky & Telescope regularly publish articles about upcoming astronomical events, including eclipses. 

I hope this helps! 


### Text generation grounded in Google Search results

Now you can add the `tools` keyword arg with a `grounding` tool of `grounding.GoogleSearchRetrieval()` to instruct the LLM to first perform a Google Search with the prompt, then construct an answer based on the web search results.

The search queries and [Search Entry Point](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points) are available for each `Candidate` in the response. The helper function `print_grounding_response()` prints the response text with citations.

In [9]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = model.generate_content(PROMPT, tools=[tool])

print_grounding_response(response)

The next total solar eclipse in the US will be on August 23, 2044. [1][2]
 This eclipse will be visible in Greenland, Canada, and parts of the United States, specifically Montana, North Dakota, and South Dakota. [1]
 

While the US will experience a partial solar eclipse on March 29, 2025, in the far northeastern reaches, the next total solar eclipse visible from the contiguous US won't happen until 2044. [3]
 After that, the next total solar eclipse viewable from the US will be in 2045. [1]


Total solar eclipses are a rare and spectacular sight. During a total solar eclipse, the Moon passes between the Sun and Earth, completely blocking the Sun's light. This allows observers in the path of totality to see the Sun's corona, which is its outer atmosphere. [2]
 

Although there are solar eclipses every year, they are only visible from a small area on Earth.  The US is fortunate to have eight total solar eclipses predicted to occur in the 21st century. This means that, on average, a total solar eclipse is visible from some part of North America approximately every twelve years. [1]
 

----
## Grounding Sources

**Web Search Queries:** ['next solar eclipse in the US']

**Search Entry Point:**
 <style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcS6TAJerbiWUDqTA2Sl47kC1xgD5D4oT-DalhiqtUxCNtrMD1uMoGPH_VMbwfnfL-Rusv9nB1CN8xH6jOEThRFOXfsS0RwmqkYN2OBjAAOEgDibHjYLARDQ_84HlYsG3qvAyxYpFNKPNRzsxR1HMjlOr0gDbFK1Pdn2JveHt3aknelRK85BBvOAefvZNN_HJgciXIIhmnGlxBhP1NqRJKU=">next solar eclipse in the US</a>
  </div>
</div>

### Grounding Chunks
1. [cbsnews.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQ8b1hLAix8HDnjKrDziAZP13fxn4odRMuPy1glEzwZQNAqbM7h-geXrXZoqL9BRmePA5jkLjfD7qTyAK_iS0NM2hvnaAIdCxs0NsPQxkL6VnZQwD7I8BUKnOCEUD5vJBJ8KXsSr00j8W69_cYsSBJCoPLMYES4WHoqo7w=)
2. [nasa.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQoVi-TSgWzKb21m1RTpeXrNoKJybwS41Ub3MTC5yb9QggpkY4o_duxDF_nv7khBAR9iaesX0YFyY7dytUTtyEFH96E8kYOYZjhW-npzkTKWFW9TTu-hzDLmOqad7rCJ50uXsIxRg5Mb80DTel23pF7WSa3pKLVYnScFQ==)
3. [space.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcTx0GmwrUYer8WOhULOpF4-2KwoT5RHOxpk5vWZYzCBgHf7b2RghQ-wVqF5Jc-XrhHOkHzc5hAYkPESafECOdTbXHGwviiHEqdquiNkGfAENdA5haIWWH3GFf2YPeyjj7uMOWpJVK4nbDU0lTe03yMkwA==)


Note that the response without grounding only has limited information from the LLM about solar eclipses. Whereas the response that was grounded in web search results contains the most up to date information from web search results that are returned as part of the LLM with grounding request.

## Example: Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest). You'll ask a question about a GoogleSQL query to create an [object table in BigQuery](https://cloud.google.com/bigquery/docs/object-table-introduction).

### Creating a data store in Vertex AI Search

In this example, you'll use a website-based data store that contains content from the Google Cloud website, including documentation.

Refer to __Task 4__ and __Task 5__ in the lab instruction manual to perform the steps advised below. Once the data store has been created, make note of the ID to input in the cells below.

Follow the tutorial steps in the Vertex AI Search documentation to:

1. [Create a data store with website data](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store) that uses the URL pattern `cloud.google.com/*` in the **Sites to include** field.
2. [Create a search app](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app) that is attached to that data store. You should also enable the **Enterprise edition features** so that you can search indexed records within the data store.

Note: The data store must be in the same project that you are using for Gemini.

__Once you've created a data store, obtain the Data Store ID and input it below.__

Note: You will need to wait for data ingestion to finish before using a data store with grounding. For more information, see [create a data store](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es).

In [ ]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "techcps_1733829237636"  # @param {type:"string"}

Now you can ask a question about object tables in BigQuery and when to use them:

In [ ]:
PROMPT = "When should I use an object table in BigQuery? And how does it store data?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [ ]:
response = model.generate_content(PROMPT)

display(Markdown(response.text))

### Text generation grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=DATA_STORE_PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)

response = model.generate_content(PROMPT, tools=[tool])
print_grounding_response(response)

Note that the response without grounding only has limited information from the LLM about object tables in BigQuery that might not be accurate. Whereas the response that was grounded in Vertex AI Search results contains the most up to date information from the Google Cloud documentation about BigQuery, along with citations of the information.

<div class="alert alert-block alert-warning">
<b>⚠️ Important notes:</b><br>
<br>
<b>If you get an error when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;In order for this sample notebook to work with data store in Vertex AI Search,<br>
&nbsp;&nbsp;&nbsp;&nbsp;you'll need to create a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store">data store</a> <b>and</b> a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app">search app</a> associated with it in Vertex AI Search.<br>
&nbsp;&nbsp;&nbsp;&nbsp;If you only create a data store, the previous request will return errors when making queries against the data store.
<br><br>
<b>If you get an empty response when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;You will need to wait for data ingestion to finish before using a data store with grounding.<br>
&nbsp;&nbsp;&nbsp;&nbsp;For more information, see <a href="https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es">create a data store</a>.
</div>
</div>

## Example: Grounded chat responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

You'll ask a question about Vertex AI and a follow up question about managed datasets in Vertex AI:

In [ ]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use?"

### Chat session without grounding

Start a chat session and send messages to the LLM with no grounding:

In [ ]:
chat = model.start_chat()

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

### Chat session grounded in Google Search results

Now you can add the `tools` keyword arg with a grounding tool of `grounding.GoogleSearchRetrieval()` to instruct the chat model to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [ ]:
chat = model.start_chat()
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = chat.send_message(PROMPT, tools=[tool])
print_grounding_response(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
print_grounding_response(response)

### Chat session grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
chat = model.start_chat()
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=DATA_STORE_PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)
response = chat.send_message(PROMPT, tools=[tool])
print_grounding_response(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
print_grounding_response(response)

## Cleaning up

To avoid incurring charges to your Google Cloud account for the resources used in this notebook, follow these steps:

1. To avoid unnecessary Google Cloud charges, use the [Google Cloud console](https://console.cloud.google.com/) to delete your project if you do not need it. Learn more in the Google Cloud documentation for [managing and deleting your project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. If you used an existing Google Cloud project, delete the resources you created to avoid incurring charges to your account. For more information, refer to the documentation to [Delete data from a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores), then delete your data store.
1. Disable the [Vertex AI Search and Conversation API](https://console.cloud.google.com/apis/api/discoveryengine.googleapis.com) and [Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com) in the Google Cloud Console.